Paleta de colores
['#f5f5f1', '#e50914', '#b20710', '#221f1f']


In [ ]:
import pandas as pd

# Intenta leer el archivo ignorando líneas con errores
df = pd.read_csv("netflix.csv")

# Guarda el resultado en un archivo Excel nuevo
df.to_excel("netflix.xlsx",index=False)

SyntaxError: invalid non-printable character U+00A0 (187287816.py, line 7)